In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def return_sugars(country):
    return worlds_sugars[country in worlds_sugars.countries].countries

In [9]:
os.getcwd()

'C:\\Users\\dsharp\\PycharmProjects\\kaggle_practice'

In [17]:
data_directory = os.getcwd()
data_path = 'data\world-food-facts\FoodFacts.csv'
data_path = os.path.join(data_directory, data_path)

if os.path.exists(data_path): 
    world_food_facts = pd.read_csv(data_path)
    world_food_facts.countries = world_food_facts.countries.str.lower()
    print("Data loaded. There are {} rows".format(world_food_facts.shape[0]))
else:
    print("Error: source data location: {} does not exist".format(data_path))

C:\Users\dsharp\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0,3,5,27,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data loaded. There are 65503 rows


In [32]:
world = world_food_facts[pd.notnull(world_food_facts.countries_tags)]

In [27]:
world_food_facts[pd.isnull(world_food_facts.countries_tags)].shape[0]

211

In [28]:
worlds_sugars = world_food_facts[world_food_facts.sugars_100g.notnull()]

In [53]:
world = world_food_facts[pd.notnull(world_food_facts.countries_tags)]

In [55]:
world.countries_tags.head()

0    en:france
1    en:france
2    en:france
3    en:france
4    en:france
Name: countries_tags, dtype: object

In [56]:
tag_index = world.columns.get_loc('countries_tags')

In [73]:
country_tags = []
for index, item in world.iterrows():
    if len(item[tag_index].split(",")) > 1:
        for county in (item[tag_index].split(",")):
            

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

In [35]:
for tag in world.groupby('countries_tags'):
    print(tag)

('To be checked,Complete,Nutrition facts completed,Ingredients completed,Expiration date completed,Characteristics completed,Photos validated,Photos uploaded',       code  url                                            creator  \
11895  NaN  NaN  villecomtal-sur-arros-gers-france,saint-just-c...   
11934  NaN  NaN                  villecomtal-sur-arros-gers-france   
12011  NaN  NaN                  villecomtal-sur-arros-gers-france   
12013  NaN  NaN                  villecomtal-sur-arros-gers-france   
12023  NaN  NaN                  villecomtal-sur-arros-gers-france   
12025  NaN  NaN                  villecomtal-sur-arros-gers-france   
12076  NaN  NaN                  villecomtal-sur-arros-gers-france   
12078  NaN  NaN                  villecomtal-sur-arros-gers-france   

                              created_t    created_datetime last_modified_t  \
11895  Villecresnes,Villeurbanne,France         Dia,Sogeres          France   
11934                   Torcy 77,France           C

In [9]:
country_list = np.unique(world.countries)

In [24]:
country_list

array(['albania', 'albania,italia', 'albania,italia, en:denmark',
       'albania,italy',
       'albanie,belgique,france,allemagne,italie,pays-bas,espagne',
       'alemania,españa', 'algeria', 'algérie', 'algérie, en:france',
       'algérie,belgique,canada,france,maroc,suisse', 'algérie,france',
       'allemagne', 'allemagne, en:denmark', 'allemagne,belgique',
       'allemagne,belgique,france', 'allemagne,france', 'allemagne,suisse',
       'andorra', 'arabie saoudite', 'argentina',
       'argentina, en:saudi-arabia', 'arménie', 'aruba', 'australia',
       'australia,france,united kingdom', 'australia,germany',
       'australia,indonesia', 'australia,indonesia,united states',
       'australia,japan', 'australia,malaysia', 'australia,new zealand',
       'australia,philippines', 'australia,united kingdom',
       'australia,united kingdom,united states', 'australia,united states',
       'australie', 'australie,belgique,france', 'australie,france',
       'australien,deutschlan

In [27]:
for country in world.countries:
    if 'japan' in country or '日本' in country:
        print(country)

日本
日本
日本
日本
日本
日本
日本,en:日本
日本
日本
日本
日本
日本
日本
日本
japan
japan
日本
日本
日本
france,japan
japan
japan
japan
日本
日本
日本
japan
japan
日本
japan
日本
日本
japan
フランス,日本
日本
日本
日本
japan
日本
japan
france,japan
日本,other:日本
japan
japan
日本
日本
australia,japan


In [10]:
type(country_list)

numpy.ndarray

In [29]:
return_sugars('france')

KeyError: False